In [1]:
from src.lobbyingDataPage import *
from src.lobbyingScraper import *

In [2]:
def convert_html_list(html_list):
    page_list = []
    for i, html in enumerate(html_list):
        print(f'converting html index {i} into data page')
        if 'Lobbyist Entity' in str(html):
            page_list.append(EntityDataPage(html))
        else:
            page_list.append(LobbyistDataPage(html))
    return page_list

In [3]:
def extract_and_save(html_list):
    page_list = convert_html_list(html_list)
    for page in page_list:
        page.save()

def pull_data(url):
    headers={"User-Agent": "Mozilla/5.0 (iPad; CPU OS 12_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148"}
    result = requests.get(url, headers=headers)
    result.raise_for_status()
    return result.content

def download_html_list(url_list):
    html_list = []
    for url in url_list:
        print("Pulling data from " + url)
        html_list.append(pull_data(url))
    return html_list

def save_data_from_url_list(url_list):
    disclosure_links = extract_and_save(download_html_list(url_list))
    html_list = download_html_list(disclosure_links)

In [4]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

def extract_client_links(year):
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    url = 'https://www.sec.state.ma.us/LobbyistPublicSearch/Default.aspx'

    driver.get(url)

    driver.find_element('id','ContentPlaceHolder1_rdbSearchByType').click()
    select = Select(driver.find_element(By.CLASS_NAME,'p3'))

    select.select_by_value(year)
    Select(driver.find_element('id','ContentPlaceHolder1_ucSearchCriteriaByType_drpType')).select_by_value('L')
    driver.find_element('id','ContentPlaceHolder1_btnSearch').click()

    find_table = driver.find_element(By.ID,'ContentPlaceHolder1_ucSearchResultByTypeAndCategory_grdvSearchResultByTypeAndCategory')
    links = find_table.find_elements(By.TAG_NAME,'a')
    links_list = [l.get_attribute('href') for l in links if str(l.get_attribute('href')).startswith('javascript') == False]
    driver.quit()
    return links_list

def extract_disclosures(list_of_links):
    disclosure_reports = []

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

    for link in list_of_links:
    # print(link)
        driver.get(link)
        all_links = driver.find_elements(By.CLASS_NAME,'BlueLinks')
        disclosure_links = [l.get_attribute('href') for l in all_links if 'CompleteDisclosure' in l.get_attribute('href')]
        for dl in disclosure_links:
            disclosure_reports.append(dl)
    driver.quit()

    return disclosure_reports


In [5]:
import pickle
with open("tests/testfiles/2005html.pkl", "rb") as f:
    html05 = pickle.load(f)
with open("tests/testfiles/2020html.pkl", "rb") as f:
    html20 = pickle.load(f)
test_htmls = html05.__add__(html20)

In [ ]:
pages = convert_html_list(test_htmls)

In [19]:
cc = [page for page in pages if "Client Compensation" in page.soup.text]
len(cc)

41

In [41]:
def divide_chunks(some_list,chunk_size):
            for i in range(0, len(some_list), chunk_size):
                yield some_list[i:i+chunk_size]

self1 = cc[0]
columns = ['Client Name','Amount']
query = re.compile(r"(?<=NameAmount).*?(?=Total salaries received)",re.DOTALL)
compensation_table = re.search(query, self1.soup.text).group()
self1.tables['Compensation'] = pd.DataFrame(columns=columns)
split_text=[line.strip() for line in compensation_table.split('\n') if line.strip()]
divided_text = list(divide_chunks(split_text, 2))
compensation_df = pd.DataFrame(divided_text,columns=columns)
self1.tables['Compensation'] = pd.concat([self1.tables['Compensation'], compensation_df])
self1.tables['Compensation']

,Client Name,Amount
0,Massachusetts Business Alliance for Education,"$6,000.00"
1,The Education Trust Inc,"$3,900.00"


In [23]:
from IPython.display import HTML

HTML(str(self1.soup.html))

In [ ]:
# Header:
# Authorizing Officer name / Lobbyist name
# Title / NA
# Business name / Business name
# Address / Address
# City, state, zip code / Citym, state, zip code
# country / country
# NA / Agent Type

#TABLES:
# Entities
# Lobbyists
# Client Compensation
# Compensation/Salaries Paid
# Activities, Bill Numbers and Titles
# Operating Expenses
# Meals, Travel, and Entertainment Expenses
# Additional Expenses
# Campaign Contributions